In [96]:
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
from nltk.corpus import movie_reviews
from tqdm import tqdm
tqdm.pandas()





In [2]:
filt = pd.read_csv("filtered data set.csv")

In [3]:
filt.head()

,Unnamed: 0,Course Key,Term Code,Comments
0,2206,200710-11026,200710,3.5 out of 5 stars
1,2207,200710-11026,200710,A lot of reading and sometimes it was boring. ...
2,2208,200710-11026,200710,"a lot of reading on your own part, but interes..."
3,2209,200710-11026,200710,"BIOS 201 is a great class,not only because of ..."
4,2210,200710-11026,200710,"Bios 201 was an organized, well taught class. ..."


In [28]:
documents = [(set(movie_reviews.words(fileid)), category)
              for category in movie_reviews.categories()
              for fileid in movie_reviews.fileids(category)]

In [29]:
movreviews = pd.DataFrame(documents)
movreviews.columns = ['allw','cat']

In [ ]:
allw = Counter(w.lower() for w in movie_reviews.words() if len(w) > 2)
topw = set(pd.Series(allw).sort_values(ascending = False).index[:2000])

In [87]:
huh = pd.read_csv("keyWords.csv")[["Happy","Unhappy"]]
topw = list(huh["Happy"].values) + list(huh["Unhappy"].values)

In [88]:
movreviews['widf'] = movreviews['allw'].apply(lambda kw: {w:(True if w in kw else False) for w in topw})
forc = movreviews[['widf','cat']]
forctuple = list(forc.itertuples(index=False, name=None))

In [89]:
train_set, test_set = forctuple[100:], forctuple[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [90]:
print(nltk.classify.accuracy(classifier, test_set))

0.88


In [91]:
filt["clist"] = filt.Comments.apply(lambda x: x.split(" "))

filtc = filt.drop("Unnamed: 0",axis = 1)
filtc['widf'] = filtc['clist'].apply(lambda kw: {w:(True if w in kw else False) for w in topw})

filtc['classified'] = filtc['widf'].apply(lambda wv: classifier.classify(wv))

In [94]:
classifier.show_most_informative_features(15)

Most Informative Features
               insulting = True              neg : pos    =     11.8 : 1.0
                   sucks = True              neg : pos    =     11.3 : 1.0
              astounding = True              pos : neg    =     11.1 : 1.0
               ludicrous = True              neg : pos    =     11.0 : 1.0
                  regard = True              pos : neg    =     10.5 : 1.0
             outstanding = True              pos : neg    =     10.4 : 1.0
          excruciatingly = True              neg : pos    =     10.0 : 1.0
              incoherent = True              neg : pos    =     10.0 : 1.0
             fascination = True              pos : neg    =      9.9 : 1.0
                seamless = True              pos : neg    =      9.3 : 1.0
                  hatred = True              pos : neg    =      9.3 : 1.0
               illogical = True              neg : pos    =      9.3 : 1.0
                  feeble = True              neg : pos    =      9.3 : 1.0

In [105]:
pd.set_option('max_colwidth', 300)
filtc[["Comments","classified"]][filtc["classified"] == "neg"].head()

,Comments,classified
0,3.5 out of 5 stars,neg
1,"A lot of reading and sometimes it was boring. Especially when there was two different people with two very different personalities where teaching it. some of the concepts were interesting, some were incredibly boring and seemed pointless. I would often say to myself that ""I dont need to know thi...",neg
2,"a lot of reading on your own part, but interesting overall",neg
3,"BIOS 201 is a great class,not only because of the material it covers, but because the class has a very relaxed atmosphere. The discussion sessions are helpful, but sometimes the preparation forms were so tedious, it didn't foster enough conversation. However it does provide a structured way for ...",neg
4,"Bios 201 was an organized, well taught class. At times a lecture method of teaching made it difficult to follow in such a large class, but mostly this was not a problem. Also, discussion sessions helped us understand certain topics in greater detail and allowed us to discuss them in a more perso...",neg


In [103]:
len(filtc[["Comments","classified"]][filtc["classified"] == "pos"])

94

In [104]:
len(filtc[["Comments","classified"]][filtc["classified"] == "neg"])

18684

In [106]:
'but' in ['buttt','butttttt']

False